In [1]:
from sklearn.linear_model  import LinearRegression  
from sklearn.metrics import mean_squared_error 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
 
sales=pd.read_csv("NYC_Sales_Samples.csv").replace("-",np.nan).dropna()
sales.iloc[:,2::]=sales.iloc[:,2::].astype("float")
sales=sales.assign(log_GSF=np.log(sales.GROSS_SQUARE_FEET), log_SP=np.log(sales.SALE_PRICE))
sales=sales.sample(n=sales.shape[0],replace=False,ignore_index=True)
sales

/tmp/ipykernel_32/615166923.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  sales.iloc[:,2::]=sales.iloc[:,2::].astype("float")


,BOROUGH,NEIGHBORHOOD,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,SALE_PRICE,log_GSF,log_SP
0,3,WINDSOR TERRACE,3.0,0.0,3.0,1260.0,4209.0,1920.0,2200000.0,8.344980,14.603968
1,3,DOWNTOWN-METROTECH,3.0,0.0,3.0,2175.0,2116.0,1910.0,1850000.0,7.657283,14.430696
2,2,WILLIAMSBRIDGE,2.0,0.0,2.0,1520.0,3852.0,2005.0,599000.0,8.256348,13.303017
3,4,ST. ALBANS,1.0,0.0,1.0,2675.0,1014.0,1960.0,460000.0,6.921658,13.038982
4,4,OZONE PARK,2.0,0.0,2.0,1700.0,2176.0,1920.0,469775.0,7.685244,13.060009
...,...,...,...,...,...,...,...,...,...,...,...
1458,3,PARK SLOPE,1.0,0.0,1.0,1700.0,2559.0,1901.0,3600000.0,7.847372,15.096444
1459,4,ST. ALBANS,2.0,0.0,2.0,2950.0,1620.0,1920.0,665000.0,7.390181,13.407542
1460,4,BEECHHURST,1.0,0.0,1.0,5000.0,1932.0,1930.0,1145000.0,7.566311,13.950915
1461,5,NEW SPRINGVILLE,2.0,0.0,2.0,4200.0,2550.0,1970.0,743000.0,7.843849,13.518451


## Q1

* Regress log(SALES_PRICE) on log(GROSS_SQUARE_FEET) 

* Predict the outcome 



In [3]:
model=LinearRegression()
model.fit(sales[["log_GSF"]],sales["log_SP"])
predicted=model.predict(sales[["log_GSF"]])
mean_squared_error(sales["log_SP"], predicted)

0.3704118244228974

## Q2

Use cross validation to choose the best model 

Model 1: log(SALES_PRICE) on log(GROSS_SQUARE_FEET) 

Model 2: log(SALES_PRICE) on log(GROSS_SQUARE_FEET) and noise1, noise2, noise3

In [4]:
#generating random numbers and assigned to noise  
sales=sales.assign(noise1=np.random.rand(sales.shape[0]),noise2=np.random.rand(sales.shape[0]),noise3=np.random.rand(sales.shape[0]))

In [5]:
# MSE always lower (favor more complex model without cross validation)
# more noise always decrease MSE 
model=LinearRegression()
List1=["log_GSF","noise1","noise2","noise3"]
model.fit(sales[List1],sales["log_SP"])
predicted=model.predict(sales[List1])
mean_squared_error(sales["log_SP"], predicted)

0.369189164181287

In [6]:
# split data to training, validation and testing 
Train=sales[0:1000:]
Valid=sales[1000:1200:]
Test=sales[1200::]

In [7]:
# simple model better
# first model
model=LinearRegression()
model.fit(Train[["log_GSF"]],Train["log_SP"])
predicted=model.predict(Valid[["log_GSF"]])
mean_squared_error(Valid["log_SP"], predicted)

0.2642656448591363

In [8]:
# Second model
model=LinearRegression()
model.fit(Train[List1],Train["log_SP"])
predicted=model.predict(Valid[List1])
mean_squared_error(Valid["log_SP"], predicted)

0.26805331171116087

In [9]:
Train_Valid = pd.concat ([Train, Valid])
model.fit(Train_Valid[["log_GSF"]], Train_Valid["log_SP"])
predicted=model.predict(Valid[["log_GSF"]])
mean_squared_error(predicted, Valid["log_SP"]) # train better model

# us

0.2616276739952172